### Zmod template

This notebook can be used with the [template](https://github.com/pavel-demin/usb104-a7-notes/tree/master/projects/template) project that contains the USB interface connected to the ADC interface via an intermediate FIFO buffer. It requires the Zmod Digitizer module to be connected to the ZMOD A connector of the USB104 A7 board and it shows how to use basic Python scripts to read ADC samples.

Import the required libraries and instantiate `PyhubJTAG` and `PyhubIO`.

In [ ]:
from pyhubio import PyhubJTAG, PyhubIO
import numpy as np
import pylab as pl
import time

jtag = PyhubJTAG()
io = PyhubIO()

Program the FPGA.

In [ ]:
jtag.program("template.bit")

Start I/O.

In [ ]:
io.start()
io.flush()

Configure ADC.

In [ ]:
adc_cfg = [
    0x00003C,
    0x000503,
    0x000803,
    0x000800,
    0x000501,
    0x001431,
    0x000502,
    0x001425,
    0x000500,
    0x002A00,
]
time.sleep(0.1)
io.write(np.uint32(adc_cfg), 2, 0)
time.sleep(0.1)

Initialize variables.

In [ ]:
size = 131072
buffer = np.zeros(size * 2, np.int16)
status = np.zeros(1, np.uint32)

Reset the FIFO buffer, wait until the FIFO buffer is full, then read ADC samples from the FIFO buffer.

In [ ]:
io.edge(0, 1, True, 0)

io.read(status, 1, 0)
while status[0] < size:
    time.sleep(0.1)
    io.read(status, 1, 0)

io.read(buffer, 2, 0)

Plot the ADC samples separately for two channels.

In [ ]:
ch1 = buffer[0::2]
ch2 = buffer[1::2]

pl.figure(figsize=[8, 4], dpi=150, constrained_layout=True)

pl.plot(ch1)
pl.plot(ch2)

pl.xlabel("sample number")
pl.ylabel("ADC units")

pl.ylim(-9000, 9000)
pl.grid()

pl.show()

Stop I/O.

In [ ]:
io.stop()